In [1]:
# ============================
# 0. Imports & global config
# ============================
import os
import sys
import json
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, average_precision_score,
    PrecisionRecallDisplay
)

import xgboost as xgb
from xgboost import XGBClassifier

pd.options.future.infer_string = True

RANDOM_STATE = 42
DATA_PATH = "AmesHousing.csv"   # fișierul original (TSV)
ART_DIR = "L2_artifacts"        # aici salvăm split-urile curate
os.makedirs(ART_DIR, exist_ok=True)

def simple_mode(s: pd.Series):
    m = s.mode(dropna=True)
    return m.iloc[0] if len(m) else np.nan

# Clips negative numeric values (area/count columns) to zero.
def enforce_nonnegative(df: pd.DataFrame) -> pd.DataFrame:
    """Clip logical non-negative numerics at 0 (defensive net)."""
    nonneg = [
        'Lot Area','Lot Frontage','Gr Liv Area',
        'BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF','Total Bsmt SF',
        'Garage Area','Garage Cars','1st Flr SF','2nd Flr SF','Low Qual Fin SF',
        'Wood Deck SF','Open Porch SF','Enclosed Porch','3Ssn Porch','Screen Porch',
        'Pool Area','SalePrice'
    ]
    for c in nonneg:
        if c in df.columns:
            df[c] = df[c].clip(lower=0)
    return df

def normalize_categoricals(df: pd.DataFrame) -> pd.DataFrame:
    """Trim/standardize string categories (avoids hidden ‘NaN ’, ‘ y ’, etc.)."""
    obj_cols = df.select_dtypes(include='object').columns
    for c in obj_cols:
        df[c] = df[c].astype(str).str.strip()
        df.loc[df[c].isin(['', 'nan', 'None', 'NaT']), c] = np.nan
    # Central Air should be Y/N
    if 'Central Air' in df.columns:
        df['Central Air'] = df['Central Air'].replace({'Yes': 'Y', 'No': 'N', 'y': 'Y', 'n': 'N'})
    return df

def apply_absence_labels(df: pd.DataFrame) -> pd.DataFrame:
    na_none_map = {
        'Alley': 'NoAlley',
        'Bsmt Qual': 'NoBasement','Bsmt Cond': 'NoBasement','Bsmt Exposure': 'NoBasement',
        'BsmtFin Type 1': 'NoBasement','BsmtFin Type 2': 'NoBasement',
        'Fireplace Qu': 'NoFireplace',
        'Garage Type': 'NoGarage','Garage Finish': 'NoGarage','Garage Qual': 'NoGarage','Garage Cond': 'NoGarage',
        'Pool QC': 'NoPool','Fence': 'NoFence','Misc Feature': 'NoFeature'
    }
    for col, val in na_none_map.items():
        if col in df.columns:
            df[col] = df[col].replace('NA', val).fillna(val)
    return df

def fix_basement_consistency(df: pd.DataFrame) -> pd.DataFrame:
    parts = ['BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF']; total = 'Total Bsmt SF'
    has_bsmt_area = (df[parts].fillna(0).sum(axis=1) > 0)
    bad_expo = has_bsmt_area & (df['Bsmt Exposure'] == 'NoBasement')
    df['Fix_BsmtExposure'] = 0
    df.loc[bad_expo, 'Bsmt Exposure'] = 'No'
    df.loc[bad_expo, 'Fix_BsmtExposure'] = 1

    sum_parts = df[parts].fillna(0).sum(axis=1)
    all_parts_na = df[parts].isna().all(axis=1)
    total_na = df[total].isna()

    mask_true_mismatch = (~all_parts_na) & (~total_na) & (df[total] != sum_parts)
    mask_total_missing = (~all_parts_na) & total_na
    df.loc[mask_true_mismatch, total] = sum_parts[mask_true_mismatch]
    df.loc[mask_total_missing, total] = sum_parts[mask_total_missing]

    mask_parts_missing = all_parts_na & (~total_na)
    for c in parts:
        df.loc[mask_parts_missing, c] = df.loc[mask_parts_missing, c].fillna(0)

    mask_all_missing = all_parts_na & total_na & (df['Bsmt Qual'] == 'NoBasement')
    for c in parts + [total]:
        df.loc[mask_all_missing, c] = 0
    return df

def consolidate_garage_absence(df: pd.DataFrame) -> pd.DataFrame:
    """If no garage (type/area/cars imply absence), zero numerics + set labels to NoGarage."""
    g_zero = (df.get('Garage Cars', 0).fillna(0) == 0) & (df.get('Garage Area', 0).fillna(0) == 0)
    if 'Garage Type' in df.columns:
        g_abs = (df['Garage Type'] == 'NoGarage') | g_zero
        for c in ['Garage Finish','Garage Qual','Garage Cond','Garage Type']:
            if c in df.columns:
                df.loc[g_abs, c] = 'NoGarage'
        for c in ['Garage Yr Blt','Garage Area','Garage Cars']:
            if c in df.columns:
                df.loc[g_abs, c] = 0
    return df

def apply_time_filters(df: pd.DataFrame) -> pd.DataFrame:
    df = df[(df['Yr Sold'] >= df['Year Built']) & (df['Yr Sold'] >= df['Year Remod/Add'])].copy()
    df = df[df['Year Remod/Add'] >= df['Year Built']].copy()
    mask_gyear_ok = (~df['Garage Yr Blt'].notna()) | (
        (df['Garage Yr Blt'] >= 1880) & (df['Garage Yr Blt'] <= (df['Yr Sold'] + 1))
    )
    return df[mask_gyear_ok].copy()

def add_binary_flags(df: pd.DataFrame) -> pd.DataFrame:
    df['HasPool']   = (df['Pool Area'] > 0).astype(int)
    df['HasFire']   = (df['Fireplaces'] > 0).astype(int)
    df['HasAC']     = (df['Central Air'] == 'Y').astype(int)
    df['HasFence']  = (df['Fence'] != 'NoFence').astype(int)
    df['HasGarage'] = (df['Garage Cars'].fillna(0) > 0).astype(int)
    df['AmenitiesCount'] = df[['HasPool','HasFire','HasAC','HasFence','HasGarage']].sum(axis=1)
    return df

def drop_low_signal_columns(df: pd.DataFrame) -> pd.DataFrame:
    to_drop = ['Utilities','Street','Condition 2','Roof Matl','Heating','Pool QC','Misc Feature','Garage Cond']
    keep = [c for c in to_drop if c in df.columns]
    return df.drop(columns=keep).copy()

def ordinal_encode_inplace(df: pd.DataFrame) -> pd.DataFrame:
    ord_orders = {
        'Exter Qual':['Po','Fa','TA','Gd','Ex'],'Exter Cond':['Po','Fa','TA','Gd','Ex'],
        'Bsmt Qual':['NoBasement','Po','Fa','TA','Gd','Ex'],'Bsmt Cond':['NoBasement','Po','Fa','TA','Gd','Ex'],
        'Bsmt Exposure':['NoBasement','No','Mn','Av','Gd'],
        'BsmtFin Type 1':['NoBasement','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],
        'BsmtFin Type 2':['NoBasement','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],
        'Heating QC':['Po','Fa','TA','Gd','Ex'],'Electrical':['FuseP','FuseF','Mix','FuseA','SBrkr'],
        'Kitchen Qual':['Po','Fa','TA','Gd','Ex'],'Fireplace Qu':['NoFireplace','Po','Fa','TA','Gd','Ex'],
        'Garage Finish':['NoGarage','Unf','RFn','Fin'],'Garage Qual':['NoGarage','Po','Fa','TA','Gd','Ex'],
        'Paved Drive':['N','P','Y'],'Lot Shape':['IR3','IR2','IR1','Reg'],'Land Slope':['Sev','Mod','Gtl'],
    }
    for col, order in ord_orders.items():
        if col in df.columns:
            cat = pd.Categorical(df[col], categories=order, ordered=True)
            s = pd.Series(cat.codes, index=df.index).astype('float64')
            s[s == -1] = np.nan
            df[col] = s
    return df

def bucket_rare(df: pd.DataFrame, cols: list, min_count: int = 5) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            s = df[c].astype(str)  # TODO check--> edge case
            vc = df[c].value_counts(dropna=False)
            rare = set(vc[vc < min_count].index)
            df[c] = df[c].where(~df[c].isin(rare), "__Other__")
    return df

In [2]:
def compute_train_stats(df_tr: pd.DataFrame):
    stats = {}
    if 'Lot Frontage' in df_tr.columns and 'Neighborhood' in df_tr.columns:
        stats['lf_med_nb']   = df_tr.groupby('Neighborhood')['Lot Frontage'].median()
        stats['lf_med_glob'] = df_tr['Lot Frontage'].median()
    else:
        stats['lf_med_nb'] = pd.Series(dtype='float64'); stats['lf_med_glob'] = np.nan

    label_cols_all = ['Garage Finish','Garage Qual','Garage Cond']
    label_cols = [c for c in label_cols_all if c in df_tr.columns]
    g_has_tr = (
        (df_tr.get('Garage Type', 'NoGarage') != 'NoGarage') |
        (df_tr.get('Garage Area', pd.Series(0, index=df_tr.index)).fillna(0) > 0) |
        (df_tr.get('Garage Cars', pd.Series(0, index=df_tr.index)).fillna(0) > 0)
    )
    if 'Garage Type' in df_tr.columns and any(label_cols):
        try:
            stats['g_modes_by_type'] = df_tr[g_has_tr].groupby('Garage Type')[label_cols].agg(simple_mode)
        except Exception:
            stats['g_modes_by_type'] = pd.DataFrame()
    else:
        stats['g_modes_by_type'] = pd.DataFrame()

    stats['g_med_area_by_type'] = (df_tr[g_has_tr].groupby('Garage Type')['Garage Area'].median()
                                   if {'Garage Type','Garage Area'} <= set(df_tr.columns) else pd.Series(dtype='float64'))
    stats['g_med_cars_by_type'] = (df_tr[g_has_tr].groupby('Garage Type')['Garage Cars'].median()
                                   if {'Garage Type','Garage Cars'} <= set(df_tr.columns) else pd.Series(dtype='float64'))

    if {'Mas Vnr Area','Mas Vnr Type'} <= set(df_tr.columns):
        ref = df_tr[(df_tr['Mas Vnr Area'].fillna(0) > 0) & df_tr['Mas Vnr Type'].notna() & (df_tr['Mas Vnr Type'] != 'None')]
        stats['mvt_nb_mode'] = (ref.groupby('Neighborhood')['Mas Vnr Type'].agg(simple_mode)
                                if 'Neighborhood' in df_tr.columns else pd.Series(dtype='object'))
        gmode = df_tr['Mas Vnr Type'].dropna().mode()
        stats['mvt_global_mode'] = gmode.iloc[0] if len(gmode) else 'BrkFace'
    else:
        stats['mvt_nb_mode'] = pd.Series(dtype='object'); stats['mvt_global_mode'] = 'BrkFace'

    num_cols = [c for c in df_tr.select_dtypes(include=[np.number]).columns if c != 'SalePrice']
    stats['num_medians'] = df_tr[num_cols].median(numeric_only=True)
    return stats

def apply_train_stats(df_block: pd.DataFrame, stats: dict) -> pd.DataFrame:
    if {'Lot Frontage','Neighborhood'} <= set(df_block.columns) and not pd.isna(stats.get('lf_med_glob', np.nan)):
        df_block['Lot Frontage'] = df_block['Lot Frontage'].fillna(df_block['Neighborhood'].map(stats['lf_med_nb'])).fillna(stats['lf_med_glob'])

    g_has = (
        (df_block.get('Garage Type', 'NoGarage') != 'NoGarage') |
        (df_block.get('Garage Area', pd.Series(0, index=df_block.index)).fillna(0) > 0) |
        (df_block.get('Garage Cars', pd.Series(0, index=df_block.index)).fillna(0) > 0)
    )
    if isinstance(stats.get('g_modes_by_type'), pd.DataFrame) and not stats['g_modes_by_type'].empty and 'Garage Type' in df_block.columns:
        need_fix = g_has & (
            (df_block.get('Garage Finish', pd.Series(np.nan, index=df_block.index)).isna()) |
            (df_block.get('Garage Qual',   pd.Series(np.nan, index=df_block.index)).isna()) |
            (df_block.get('Garage Cond',   pd.Series(np.nan, index=df_block.index)).isna()) |
            (df_block.get('Garage Finish', pd.Series('NoGarage', index=df_block.index)) == 'NoGarage') |
            (df_block.get('Garage Qual',   pd.Series('NoGarage', index=df_block.index)) == 'NoGarage') |
            (df_block.get('Garage Cond',   pd.Series('NoGarage', index=df_block.index)) == 'NoGarage')
        )
        for c in ['Garage Finish','Garage Qual','Garage Cond']:
            if c in df_block.columns and c in stats['g_modes_by_type'].columns:
                fill_vals = df_block.loc[need_fix, 'Garage Type'].map(stats['g_modes_by_type'][c])
                df_block.loc[need_fix, c] = df_block.loc[need_fix, c].fillna(fill_vals)

    if {'Garage Area','Garage Type'} <= set(df_block.columns) and not stats.get('g_med_area_by_type', pd.Series()).empty:
        m_area_na = g_has & df_block['Garage Area'].isna()
        df_block.loc[m_area_na, 'Garage Area'] = df_block.loc[m_area_na, 'Garage Type'].map(stats['g_med_area_by_type'])
    if {'Garage Cars','Garage Type'} <= set(df_block.columns) and not stats.get('g_med_cars_by_type', pd.Series()).empty:
        m_cars_na = g_has & df_block['Garage Cars'].isna()
        df_block.loc[m_cars_na, 'Garage Cars'] = df_block.loc[m_cars_na, 'Garage Type'].map(stats['g_med_cars_by_type'])

    if {'Mas Vnr Area','Mas Vnr Type'} <= set(df_block.columns):
        area = df_block['Mas Vnr Area']; typ = df_block['Mas Vnr Type']
        m_area0 = area.fillna(0) == 0; m_area_pos = area.fillna(0) > 0
        m_type_none = typ == 'None'; m_type_na = typ.isna()
        df_block.loc[m_type_na & m_area0, 'Mas Vnr Type'] = 'None'
        df_block.loc[(~m_type_none & ~m_type_na) & m_area0, 'Mas Vnr Type'] = 'None'
        mask_pos_missing = m_area_pos & (m_type_none | m_type_na)
        nb_mode = stats.get('mvt_nb_mode', pd.Series(dtype='object')); global_mode = stats.get('mvt_global_mode', 'BrkFace')
        if not nb_mode.empty and 'Neighborhood' in df_block.columns:
            nb_fill = df_block.loc[mask_pos_missing, 'Neighborhood'].map(nb_mode).fillna(global_mode)
        else:
            nb_fill = pd.Series(global_mode, index=df_block.index).loc[mask_pos_missing]
        df_block.loc[mask_pos_missing, 'Mas Vnr Type'] = nb_fill
    return df_block

def fill_not_present_numerics(df_block: pd.DataFrame) -> pd.DataFrame:
    exist_num = ['Garage Yr Blt','Garage Area','Garage Cars',
                 'BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF','Total Bsmt SF',
                 'Bsmt Full Bath','Bsmt Half Bath','Pool Area','Mas Vnr Area']
    for c in exist_num:
        if c in df_block.columns:
            df_block[f'{c}_was_missing'] = df_block[c].isna().astype(int)
            df_block[c] = df_block[c].fillna(0)
    return df_block

def fill_remaining_numerics_with_train_median(df_block: pd.DataFrame, num_medians: pd.Series) -> pd.DataFrame:
    for c in df_block.select_dtypes(include=[np.number]).columns:
        if df_block[c].isna().any():
            df_block[c] = df_block[c].fillna(num_medians.get(c, df_block[c].median()))
    return df_block

In [3]:
def generate_sanity_report(df_raw, df_train_clean, df_test_clean, meta, make_plots=True):
    df_clean_all = pd.concat([df_train_clean, df_test_clean], ignore_index=True)
    n_raw = meta.get('n_raw', len(df_raw)); n_after_time = meta.get('n_after_time', len(df_clean_all))
    dropped_time = n_raw - n_after_time; pct_dropped = (dropped_time / n_raw) * 100 if n_raw else 0
    print("=== Sanity report ===")
    print(f"- Rows initiale: {n_raw}")
    print(f"- După filtre temporale: {n_after_time}  (drop: {dropped_time} | {pct_dropped:.2f}%)")
    n_fix_bsmt = meta.get('n_fix_bsmt_exposure', int(df_clean_all.get('Fix_BsmtExposure', pd.Series(0)).sum()))
    print(f"- Corecții Bsmt Exposure (NoBasement -> No când există arii): {n_fix_bsmt}")

    parts = ['BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF']; total = 'Total Bsmt SF'
    if all(c in df_clean_all.columns for c in parts+[total]):
        mismatch = (df_clean_all[total] != df_clean_all[parts].fillna(0).sum(axis=1)).sum()
        print(f"- Identități subsol (Total == sum(parts)) → mismatches: {mismatch}")
    else:
        print("- Identități subsol: coloane lipsă, skip.")

    na_raw = df_raw.isna().sum().sort_values(ascending=False)
    na_clean = df_clean_all.isna().sum().sort_values(ascending=False)
    print("\nTop 10 coloane cu NaN (înainte):"); print(na_raw.head(10))
    print("\nTop 10 coloane cu NaN (după curățare):"); print(na_clean.head(10))
    print(f"\nShapes: train={df_train_clean.shape}, test={df_test_clean.shape}")
    if 'SalePrice' in df_raw.columns:
        sp_raw = df_raw['SalePrice'].dropna(); sp_clean = df_clean_all['SalePrice'].dropna()
        print(f"SalePrice (raw):   n={sp_raw.size}, min={sp_raw.min():,.0f}, median={sp_raw.median():,.0f}, max={sp_raw.max():,.0f}")
        print(f"SalePrice (clean): n={sp_clean.size}, min={sp_clean.min():,.0f}, median={sp_clean.median():,.0f}, max={sp_clean.max():,.0f}")

    if make_plots:
        try: plot_sanity(df_raw, df_clean_all)
        except Exception as e: print(f"(plot warning) {e}")

def plot_sanity(df_raw, df_clean):
    if 'SalePrice' in df_raw.columns and 'SalePrice' in df_clean.columns:
        plt.figure(figsize=(7,4))
        plt.hist(df_raw['SalePrice'].dropna(), bins=50, alpha=0.35, label='raw')
        plt.hist(df_clean['SalePrice'].dropna(), bins=50, alpha=0.6, label='clean')
        plt.title("SalePrice: raw vs clean (hist)"); plt.xlabel("SalePrice"); plt.ylabel("count"); plt.legend(); plt.tight_layout(); plt.show()
    if all(c in df_clean.columns for c in ['Gr Liv Area','SalePrice']):
        plt.figure(figsize=(6,4))
        plt.scatter(df_clean['Gr Liv Area'], df_clean['SalePrice'], s=8, alpha=0.5)
        plt.title("SalePrice vs Gr Liv Area (clean)"); plt.xlabel("Gr Liv Area"); plt.ylabel("SalePrice"); plt.tight_layout(); plt.show()

def validate_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """Return a small table of issues (row_idx, column, rule, value) without mutating df."""
    issues = []
    def report(mask, col, rule):
        idx = df.index[mask]
        for i in idx:
            issues.append((int(i), col, rule, df.at[i, col] if col in df.columns else None))

    if {'Mo Sold','Yr Sold'} <= set(df.columns):
        report(~df['Mo Sold'].between(1,12), 'Mo Sold', 'month_out_of_range')
        report(~df['Yr Sold'].between(1800, 2100), 'Yr Sold', 'year_out_of_range')

    if {'Overall Qual','Overall Cond'} <= set(df.columns):
        report(~df['Overall Qual'].between(1,10), 'Overall Qual', 'qual_out_of_range')
        report(~df['Overall Cond'].between(1,10), 'Overall Cond', 'cond_out_of_range')

    if {'Year Built','Year Remod/Add'} <= set(df.columns):
        report(df['Year Remod/Add'] < df['Year Built'], 'Year Remod/Add', 'remod_before_built')

    if {'Garage Yr Blt','Yr Sold'} <= set(df.columns):
        gy = df['Garage Yr Blt']; ys = df['Yr Sold']
        report(gy.notna() & (gy < 1880), 'Garage Yr Blt', 'garage_too_old')
        report(gy.notna() & (gy > ys + 1), 'Garage Yr Blt', 'garage_after_sale+1')

    pos_when_present = ['Gr Liv Area','1st Flr SF','SalePrice']
    for c in pos_when_present:
        if c in df.columns:
            report(df[c] < 0, c, 'negative_value')

    return pd.DataFrame(issues, columns=['row_idx','column','rule','value'])

In [4]:
def clean_ames(csv_path: str,
               test_size: float = 0.2,
               random_state: int = 42,
               return_meta: bool = False,
               use_rare_bucket: bool = False,
               winsorize: bool = False,
               drop_exact_duplicates: bool = True,
               drop_audit_flags: bool = True):
    """
    Deterministic, leak-free cleaning. Outlier trimming is OFF by default (see train_band_filter).
    """
    df_raw = pd.read_csv(csv_path, sep="\t")
    meta = {'n_raw': len(df_raw), 'na_raw_top': df_raw.isna().sum().sort_values(ascending=False).head(10)}

    # ---- Safe pre-split steps
    df = df_raw.copy()
    df = normalize_categoricals(df)
    df = apply_absence_labels(df)
    df = fix_basement_consistency(df)
    df = consolidate_garage_absence(df)
    df = enforce_nonnegative(df)

    if drop_exact_duplicates:
        before = len(df); df = df.drop_duplicates().copy(); meta['n_dropped_dupes'] = before - len(df)

    n_before_time = len(df)
    df = apply_time_filters(df)
    meta['n_after_time'] = len(df); meta['n_dropped_time'] = n_before_time - meta['n_after_time']

    df = add_binary_flags(df)
    df = drop_low_signal_columns(df)

    if use_rare_bucket:
        nominal_candidates = ['MS SubClass','MS Zoning','Neighborhood','Condition 1','Bldg Type','House Style',
                              'Roof Style','Exterior 1st','Exterior 2nd','Mas Vnr Type','Foundation','Lot Config',
                              'Land Contour','Garage Type','Sale Type','Sale Condition']
        nominal_cols = [c for c in nominal_candidates if c in df.columns]
        df = bucket_rare(df, nominal_cols, min_count=5)

    df = ordinal_encode_inplace(df)

    # Drop IDs
    df = df.drop(columns=['Order','PID'], errors='ignore')

    # ---- Split early (avoid leakage)
    train_idx, test_idx = train_test_split(df.index, test_size=test_size, random_state=random_state)
    df_tr = df.loc[train_idx].copy(); df_te = df.loc[test_idx].copy()

    # ---- Train-only stats, apply to both
    stats = compute_train_stats(df_tr)
    df_tr = apply_train_stats(df_tr, stats); df_te = apply_train_stats(df_te, stats)

    # Not-present numerics → 0 (+indicator)
    df_tr = fill_not_present_numerics(df_tr); df_te = fill_not_present_numerics(df_te)

    # (optional) winsorization — *feature* numerics only (no target)
    if winsorize:
        num_cols_tr = [c for c in df_tr.select_dtypes(include=[np.number]).columns if c != 'SalePrice']
        q_lo = df_tr[num_cols_tr].quantile(0.01); q_hi = df_tr[num_cols_tr].quantile(0.99)
        for c in num_cols_tr:
            df_tr[c] = df_tr[c].clip(lower=q_lo.get(c, df_tr[c].min()), upper=q_hi.get(c, df_tr[c].max()))
            if c in df_te.columns:
                lo = q_lo.get(c, df_tr[c].min()); hi = q_hi.get(c, df_tr[c].max())
                df_te[c] = df_te[c].clip(lower=lo, upper=hi)

    # Remaining numerics → train medians
    df_tr = fill_remaining_numerics_with_train_median(df_tr, stats['num_medians'])
    df_te = fill_remaining_numerics_with_train_median(df_te, stats['num_medians'])

    # Reset index
    df_tr = df_tr.reset_index(drop=True); df_te = df_te.reset_index(drop=True)

    # Meta checks (report needs the flags present)
    meta['n_fix_bsmt_exposure'] = int(df.get('Fix_BsmtExposure', pd.Series(0)).sum())
    all_clean = pd.concat([df_tr, df_te], ignore_index=True)
    parts = ['BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF']; total = 'Total Bsmt SF'
    meta['post_clean_bsmt_mismatch'] = (int((all_clean[total] != all_clean[parts].fillna(0).sum(axis=1)).sum())
                                        if all(c in all_clean.columns for c in parts+[total]) else None)
    meta['na_clean_top'] = all_clean.isna().sum().sort_values(ascending=False).head(10)

    # Validation report (non-fatal)
    meta['validation_issues_train'] = validate_dataset(df_tr).head(15)
    meta['validation_issues_test']  = validate_dataset(df_te).head(15)

    # --- Drop audit flags after meta/report, if requested
    if drop_audit_flags:
        audit_cols = ['Fix_BsmtExposure', 'Fix_Garage']  # Fix_Garage may not exist
        df_tr = df_tr.drop(columns=[c for c in audit_cols if c in df_tr.columns], errors='ignore')
        df_te = df_te.drop(columns=[c for c in audit_cols if c in df_te.columns], errors='ignore')

    if return_meta:
        return df_tr, df_te, meta
    return df_tr, df_te

In [5]:
import pandas as pd
def train_band_filter(df_tr: pd.DataFrame,
                      x_col: str = 'Gr Liv Area',
                      y_col: str = 'SalePrice',
                      low_q: float = 0.10,
                      high_q: float = 0.90):
    """
    Return a row mask for TRAIN keeping only points within [low_q, high_q]
    conditional quantile band of log(y) ~ log(x). Uses sklearn QuantileRegressor.
    """
    from sklearn.linear_model import QuantileRegressor

    x = np.log1p(df_tr[x_col].clip(lower=0)).values.reshape(-1, 1)
    y = np.log1p(df_tr[y_col].clip(lower=0)).values
    qr_low  = QuantileRegressor(quantile=low_q, alpha=0).fit(x, y)
    qr_high = QuantileRegressor(quantile=high_q, alpha=0).fit(x, y)
    y_pred_lo = qr_low.predict(x); y_pred_hi = qr_high.predict(x)
    keep_mask = (y >= y_pred_lo) & (y <= y_pred_hi)
    return keep_mask

In [6]:
def find_imbalanced_columns(
    df: pd.DataFrame,
    max_unique: int = 10,
    threshold: float = 0.7
) -> pd.DataFrame:
    """
    Scanează toate coloanele din df și returnează un tabel cu acele coloane
    unde o singură clasă/valoare domină (>= threshold).
    """
    rows = []

    for col in df.columns:
        s = df[col]

        if s.notna().sum() == 0:
            continue

        vc = s.value_counts(dropna=False)

        if len(vc) > max_unique:
            continue

        total = vc.sum()
        maj_val = vc.index[0]
        maj_cnt = int(vc.iloc[0])
        maj_pct = maj_cnt / total

        if maj_pct >= threshold:
            rows.append({
                "column": col,
                "dtype": str(s.dtype),
                "n_unique": len(vc),
                "majority_value": maj_val,
                "majority_count": maj_cnt,
                "majority_pct": round(maj_pct * 100, 2),
                "value_counts": vc.to_dict()
            })

    if not rows:
        return pd.DataFrame(columns=["column","dtype","n_unique","majority_value",
                                     "majority_count","majority_pct","value_counts"])

    return pd.DataFrame(rows).sort_values("majority_pct", ascending=False)

In [7]:
# if __name__ == "__main__":
#     # Cleaning only (no outlier trimming)
#     df_train_20, df_test_20, meta_20 = clean_ames(
#         "AmesHousing.csv",
#         test_size=0.2,
#         random_state=42,
#         return_meta=True,
#         use_rare_bucket=False,   # stays OFF (safe)
#         winsorize=False,         # stays OFF (modeling choice)
#         drop_exact_duplicates=True,
#         drop_audit_flags=True    # <— recommended ON for modeling
#     )
#     print("Train 20% shape:", df_train_20.shape, "Test 20% shape:", df_test_20.shape)
#
#     # Report + quick plots
#     generate_sanity_report(
#         df_raw=pd.read_csv("AmesHousing.csv", sep="\t"),
#         df_train_clean=df_train_20,
#         df_test_clean=df_test_20,
#         meta=meta_20,
#         make_plots=True
#     )
#
#     # OPTIONAL: TRAIN-ONLY trimming (leak-free)
#     # keep_mask = train_band_filter(df_train_20, x_col='Gr Liv Area', y_col='SalePrice', low_q=0.10, high_q=0.90)
#     # df_train_band = df_train_20.loc[keep_mask].reset_index(drop=True)
#     # print(f"Band-kept rows: {keep_mask.sum()} / {len(keep_mask)} ({keep_mask.mean()*100:.1f}%)")

In [8]:

RANDOM_STATE = 42

In [9]:
# Step 0.2 — load & clean (lock splits for A and B)

DATA_PATH = "AmesHousing.csv"  # tab-separated
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

# Scenario A (80/20)
df_train_A, df_test_A, meta_A = clean_ames(
    DATA_PATH,
    test_size=0.20,
    random_state=RANDOM_STATE,
    return_meta=True,
    use_rare_bucket=False,
    winsorize=False,
    drop_exact_duplicates=True,
    drop_audit_flags=True
)

# Scenario B (90/10)
df_train_B, df_test_B, meta_B = clean_ames(
    DATA_PATH,
    test_size=0.10,
    random_state=RANDOM_STATE,
    return_meta=True,
    use_rare_bucket=False,
    winsorize=False,
    drop_exact_duplicates=True,
    drop_audit_flags=True
)

print("A — shapes:", df_train_A.shape, df_test_A.shape)
print("B — shapes:", df_train_B.shape, df_test_B.shape)

A — shapes: (2213, 89) (554, 89)
B — shapes: (2490, 89) (277, 89)


In [10]:
print("\n=== Unbalanced classes în df_train_A (80/20) ===")
imb_A = find_imbalanced_columns(df_train_A, max_unique=10, threshold=0.7)
print(imb_A.to_string(index=False))

print("\n=== Unbalanced classes în df_train_B (90/10) ===")
imb_B = find_imbalanced_columns(df_train_B, max_unique=10, threshold=0.7)
print(imb_B.to_string(index=False))


=== Unbalanced classes în df_train_A (80/20) ===
                    column   dtype  n_unique majority_value  majority_count  majority_pct                                                                                                      value_counts
                 HasGarage   int64         1              1            2213        100.00                                                                                                         {1: 2213}
 Total Bsmt SF_was_missing   int64         1              0            2213        100.00                                                                                                         {0: 2213}
   Bsmt Unf SF_was_missing   int64         1              0            2213        100.00                                                                                                         {0: 2213}
  BsmtFin SF 2_was_missing   int64         1              0            2213        100.00                                             

In [11]:
# Step 0.3 — assertions for both scenarios

def assert_no_nans(df, name):
    nans = df.isna().sum().sum()
    assert nans == 0, f"{name} still has {nans} NaNs"
    return True

def assert_same_columns(df_tr, df_te, tag):
    c1, c2 = list(df_tr.columns), list(df_te.columns)
    assert c1 == c2, f"{tag}: train/test columns differ!"
    return True

def feature_count_ok(df, approx=89):
    n = df.shape[1]
    print(f"- Feature count = {n} (expected ≈ {approx})")
    return n

print("Scenario A checks:")
assert_no_nans(df_train_A, "train_A"); assert_no_nans(df_test_A, "test_A")
assert_same_columns(df_train_A, df_test_A, "A")
nA = feature_count_ok(df_train_A)

print("\nScenario B checks:")
assert_no_nans(df_train_B, "train_B"); assert_no_nans(df_test_B, "test_B")
assert_same_columns(df_train_B, df_test_B, "B")
nB = feature_count_ok(df_train_B)

# Quick expected shape sanity (non-fatal)
print("\n(Non-fatal) Expected shape hints:")
print("- A ~ (2213, 89) train / (554, 89) test")
print("- B ~ (2490, 89) train / (277, 89) test")

print("\nAll Step 0.3 assertions passed!!")


Scenario A checks:
- Feature count = 89 (expected ≈ 89)

Scenario B checks:
- Feature count = 89 (expected ≈ 89)

(Non-fatal) Expected shape hints:
- A ~ (2213, 89) train / (554, 89) test
- B ~ (2490, 89) train / (277, 89) test

All Step 0.3 assertions passed!!


In [12]:
# Step 0.4 — save frozen splits and metadata

OUT_DIR = "L2_artifacts"
os.makedirs(OUT_DIR, exist_ok=True)

# Data splits (CSV to avoid parquet deps)
df_train_A.to_csv(os.path.join(OUT_DIR, "L2_clean_train_A.csv"), index=False)
df_test_A .to_csv(os.path.join(OUT_DIR, "L2_clean_test_A.csv"),  index=False)
df_train_B.to_csv(os.path.join(OUT_DIR, "L2_clean_train_B.csv"), index=False)
df_test_B .to_csv(os.path.join(OUT_DIR, "L2_clean_test_B.csv"),  index=False)

# Meta objects can contain Series/DataFrames; pickle them safely
with open(os.path.join(OUT_DIR, "meta_A.pkl"), "wb") as f:
    pickle.dump(meta_A, f)
with open(os.path.join(OUT_DIR, "meta_B.pkl"), "wb") as f:
    pickle.dump(meta_B, f)

# Feature registry (exclude SalePrice for now; PriceClass will be created in Step 1)
feature_names = [c for c in df_train_A.columns if c != "SalePrice"]
with open(os.path.join(OUT_DIR, "feature_names.json"), "w", encoding="utf-8") as f:
    json.dump(sorted(feature_names), f, indent=2)

# Nice one-screen summary
def rows_after_filters(meta):
    return meta.get("n_after_time", None)

summary = {
    "random_state": RANDOM_STATE,
    "scenario_A": {
        "train_shape": df_train_A.shape,
        "test_shape":  df_test_A.shape,
        "n_features":  df_train_A.shape[1],
        "rows_after_temporal_filters": rows_after_filters(meta_A),
    },
    "scenario_B": {
        "train_shape": df_train_B.shape,
        "test_shape":  df_test_B.shape,
        "n_features":  df_train_B.shape[1],
        "rows_after_temporal_filters": rows_after_filters(meta_B),
    }
}
with open(os.path.join(OUT_DIR, "step0_summary.json"), "w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2)

print("Saved:")
for fn in [
    "L2_clean_train_A.csv","L2_clean_test_A.csv",
    "L2_clean_train_B.csv","L2_clean_test_B.csv",
    "meta_A.pkl","meta_B.pkl",
    "feature_names.json","step0_summary.json"
]:
    print(" -", os.path.join(OUT_DIR, fn))

print("\nStep 0 is complete and splits are FROZEN.")

Saved:
 - L2_artifacts\L2_clean_train_A.csv
 - L2_artifacts\L2_clean_test_A.csv
 - L2_artifacts\L2_clean_train_B.csv
 - L2_artifacts\L2_clean_test_B.csv
 - L2_artifacts\meta_A.pkl
 - L2_artifacts\meta_B.pkl
 - L2_artifacts\feature_names.json
 - L2_artifacts\step0_summary.json

Step 0 is complete and splits are FROZEN.


In [13]:
# Step 0.5 — compact summary printout

def compact_meta(meta):
    n_raw = meta.get("n_raw", None)
    n_after = meta.get("n_after_time", None)
    dropped = None if (n_raw is None or n_after is None) else (n_raw - n_after)
    pct = None if (n_raw in (None, 0) or n_after is None) else 100.0 * dropped / n_raw
    return f"raw={n_raw}, after_filters={n_after}, dropped={dropped} ({pct:.2f}%)"

print("Scenario A:", summary["scenario_A"])
print("Meta A:", compact_meta(meta_A))

print("\nScenario B:", summary["scenario_B"])
print("Meta B:", compact_meta(meta_B))

print("\nNote: From now on, always reload from L2_artifacts/* for fair comparisons.")

Scenario A: {'train_shape': (2213, 89), 'test_shape': (554, 89), 'n_features': 89, 'rows_after_temporal_filters': 2767}
Meta A: raw=2930, after_filters=2767, dropped=163 (5.56%)

Scenario B: {'train_shape': (2490, 89), 'test_shape': (277, 89), 'n_features': 89, 'rows_after_temporal_filters': 2767}
Meta B: raw=2930, after_filters=2767, dropped=163 (5.56%)

Note: From now on, always reload from L2_artifacts/* for fair comparisons.


In [14]:
# Step 1.0 — config & load frozen splits
ART_IN  = "L2_artifacts"   # from Step 0
ART_OUT = "L3_artifacts"   # new folder for Step 1
os.makedirs(ART_OUT, exist_ok=True)

# Load Step 0 splits (Scenario A = 80/20, Scenario B = 90/10)
df_train_A = pd.read_csv(os.path.join(ART_IN, "L2_clean_train_A.csv"))
df_test_A  = pd.read_csv(os.path.join(ART_IN, "L2_clean_test_A.csv"))
df_train_B = pd.read_csv(os.path.join(ART_IN, "L2_clean_train_B.csv"))
df_test_B  = pd.read_csv(os.path.join(ART_IN, "L2_clean_test_B.csv"))

In [15]:
# sp = df_train_A["SalePrice"]
#
# print("SalePrice min / median / max:", sp.min(), sp.median(), sp.max())
#
# schemes = {
#     "price_3_v1_150_250": [0, 150000, 250000, sp.max()+1],
#     "price_3_v2_140_220": [0, 140000, 220000, sp.max()+1],
#     "price_3_v3_160_260": [0, 160000, 260000, sp.max()+1],
#     "price_4_v1_140_200_300": [0, 140000, 200000, 300000, sp.max()+1],
#     "price_4_v2_150_220_300": [0, 150000, 220000, 300000, sp.max()+1],
# }
#
# def summarize_scheme(name, bins):
#     tmp = df_train_A.copy()
#     tmp[name] = pd.cut(sp, bins=bins, include_lowest=True, labels=False)
#
#     vc = tmp[name].value_counts().sort_index()
#     pct = tmp[name].value_counts(normalize=True).sort_index() * 100
#
#     print(f"\n=== Schema: {name} ===")
#     print("Bin edges:", [int(b) for b in bins])
#     print("\nCounts & %:")
#     print(pd.DataFrame({
#         "count": vc,
#         "pct": pct.round(2)
#     }))
#
#     print("\nPrice range per class (TRAIN_A):")
#     grp = tmp.groupby(name)["SalePrice"].agg(["count","median","min","max"]).astype(int)
#     print(grp)
#
# for name, bins in schemes.items():
#     summarize_scheme(name, bins)